In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [6]:
def def_final(d1, d2, d3, d4, d5, d6, d7, d8):
    data = pd.read_csv('data_fin_.csv',encoding = 'cp949')
    user = pd.read_csv('user_data.csv',encoding = 'cp949')
    user['rating'] = [d1, d2, d3, d4, d5, d6, d7, d8]
    new = data[['userid','courseid','rating']]
    user_new = user[['userid','courseid','rating']]
    new = new.append(user_new)
    data_matrix = new.pivot_table('rating', index = 'userid', columns = 'courseid')
    data_matrix = data[['userid','course','rating']]
    user_matrix = user[['userid','course','rating']]
    data_matrix = data_matrix.append(user_matrix)
    rating_matrix = data_matrix.pivot_table('rating', index = 'userid', columns = 'course')
    rating_matrix = rating_matrix.fillna(0)
    rating_matrix_T = rating_matrix.transpose()
    item_sim = cosine_similarity(rating_matrix_T, rating_matrix_T)
    item_sim_df = pd.DataFrame(data = item_sim, index = rating_matrix.columns, columns = rating_matrix.columns)
    def predict_rating(rating_arr, item_sim_arr):
        rating_pred = rating_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis = 1)])
        return rating_pred
    rating_pred = predict_rating(rating_matrix.values, item_sim_df.values)
    rating_pred_matrix = pd.DataFrame(data = rating_pred, index = rating_matrix.index, columns = rating_matrix.columns)
    def get_mse(pred, actual): #사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 구하기
        pred = pred[actual.nonzero()].flatten() #평점이 있는 실제 영화만 추출
        actual = actual[actual.nonzero()].flatten()
        return mean_squared_error(pred, actual)
    def predict_rating_topsim(rating_arr, item_sim_arr, n = 20):
        pred = np.zeros(rating_arr.shape) #예측 행렬 초기화
    
        for col in range(rating_arr.shape[1]): #사용자-아이템 평점 행렬 열 크기만큼 루프 수행
            top_n_items = [np.argsort(item_sim_arr[:, col])[: -n-1 : -1]] #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬 인덱스 반환
            for row in range(rating_arr.shape[0]): #개인화된 예측 평점 계산
                pred[row, col] = item_sim_arr[col, :][top_n_items].dot(rating_arr[row, :][top_n_items].T)
                pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
        return pred
    rating_pred = predict_rating_topsim(rating_matrix.values, item_sim_df.values, n = 20)
    rating_pred_matrix = pd.DataFrame(data = rating_pred, index = rating_matrix.index, columns = rating_matrix.columns)
    user_rating_id = rating_matrix.loc[8, :]
    def get_unseen_course1(rating_matrix, userid): #이미 rating 준 코스 제외하고 추천할 수 있도록, rating 안준 코스 반환 함수
        user_rating = rating_matrix.loc[userid, :] #반환된 user_rating은 코스를 index로 가지는 Series 객체
        already_go = user_rating[ user_rating > 0 ].index.tolist()
        course_list = rating_matrix.columns.tolist()
        un_list = [ course for course in course_list if course not in already_go ]
        return un_list
    def recomm_course_by_userid1(pred_df, userid, un_list, top_n):
        # unseen course 코스를 추출해 가장 예측 평점 높은 순으로 정렬
        recomm_course = pred_df.loc[userid, un_list].sort_values(ascending = False)[: top_n]
        return recomm_course
    un_list = get_unseen_course1(rating_matrix, 8)
    recomm_course = recomm_course_by_userid1(rating_pred_matrix, 8, un_list, 30)
    recomm_course = pd.DataFrame(data = recomm_course.values, index = recomm_course.index, columns = ['pred_score'])
    def get_rmse(R, P, Q, non_zeros): #실제 R 행렬과 예측 행렬의 오차 구하기
        error = 0
        full_pred_matrix = np.dot(P, Q.T) #두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성(np.dot함수는 내적 함수)
        x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
        y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
        R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
        full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
        mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
        rmse = np.sqrt(mse)
        return rmse
    def matrix_factorization(R, K, steps, learning_rate, r_lambda):
        #R : 원본 사용자-아이템 평점 행렬
        #K : 잠재요인의 차원 수
        #step : SGD의 반복 횟수
        num_users, num_items = R.shape
        np.random.seed(1)
        P = np.random.normal(scale = 1./K, size = (num_users, K))
        Q = np.random.normal(scale = 1./K, size = (num_items, K))
    
        prev_rmse = 10000
        break_count = 0
    
        non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]
     
        for step in range(steps):
            for i, j, r in non_zeros:
                eij = r - np.dot(P[i, :], Q[j, :].T)
                P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
                Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
            rmse = get_rmse(R, P, Q, non_zeros)
            if(step % 10) == 0:
                print("### iteration step : ",step, " rmse : ", rmse)
        return P,Q
    rating = new[['userid', 'courseid', 'rating']]
    rating_matrix = rating.pivot_table('rating', index = 'userid', columns = 'courseid')
    rating_matrix = data_matrix.pivot_table('rating', index = 'userid', columns = 'course')
    P, Q = matrix_factorization(rating_matrix.values, 50, 200, 0.01, 0.01)
    pred_matrix = np.dot(P, Q.T)
    rating_pred_matrix = pd.DataFrame(data = pred_matrix, index = rating_matrix.index, columns = rating_matrix.columns)
    def get_unseen_course2(rating_matrix, userid): #이미 rating 준 코스 제외하고 추천할 수 있도록, rating 안준 코스 반환 함수
        user_rating = rating_matrix.loc[userid, :] #반환된 user_rating은 코스를 index로 가지는 Series 객체
        already_go = user_rating[ user_rating > 0 ].index.tolist()
        course_list = rating_matrix.columns.tolist()
        un_list = [ course for course in course_list if course not in already_go ]
        return un_list
    def recomm_course_by_userid2(pred_df, userid, un_list, top_n):
        # unseen course 코스를 추출해 가장 예측 평점 높은 순으로 정렬
        recomm_course = pred_df.loc[userid, un_list].sort_values(ascending = False)[: top_n]
        return recomm_course
    un_list = get_unseen_course2(rating_matrix, 8)
    matrix_recomm_course = recomm_course_by_userid2(rating_pred_matrix, 8, un_list, 30)
    matrix_recomm_course = pd.DataFrame(data = matrix_recomm_course.values, index = matrix_recomm_course.index, columns = ['pred_score'])
    course_df = data.drop(['userid', 'rating'], axis = 1)
    user_df = user[['course','rating']]
    inputId = course_df[course_df['course'].isin(user_df['course'].tolist())]
    result = pd.merge(inputId, user_df)
    result_df = result[['courseid', 'course', 'rating']]
    result_detail = result[['specific', 'water', 'tree', 'ruins', 'eat', 'hotel', 'cafe', 'pet', 'spring', 'summer', 'fall', 'winter']]
    userProfile = result_detail.transpose().dot(result_df['rating'])
    courseTable = course_df.set_index(course_df['courseid'])
    courseTable = courseTable[['specific', 'water', 'tree', 'ruins', 'eat', 'hotel', 'cafe', 'pet', 'spring', 'summer', 'fall', 'winter']]
    recoTable_df = ((courseTable * userProfile).sum(axis = 1)) / (userProfile.sum())
    recoTable_df = recoTable_df.sort_values(ascending = False) # 내림차순
    rank = course_df.loc[course_df['courseid'].isin(recoTable_df.head(20).keys())]
    rankId = data[data['course'].isin(rank['course'].tolist())]
    final = pd.merge(rankId, rank)
    fi_result = final[['courseid', 'course', 'rating']].sort_values('rating', ascending = False)
    fi_result = fi_result.drop_duplicates('courseid', keep='first')
    fi_result.to_csv("content_based.csv")
    recomm_course.to_csv("item_based.csv")
    matrix_recomm_course.to_csv("matrix_factorization.csv")
    item_based = pd.read_csv('item_based.csv', encoding = 'utf-8')
    matrix_factorization = pd.read_csv('matrix_factorization.csv', encoding = 'utf-8')
    dupli = item_based[item_based['course'].isin(matrix_factorization['course'].tolist())]
    return dupli

In [7]:
def_final(0.5, 1, 2, 3, 5, 7, 8, 10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


### iteration step :  0  rmse :  7.81819849360074
### iteration step :  10  rmse :  0.8416998318075539
### iteration step :  20  rmse :  0.41254228991819186
### iteration step :  30  rmse :  0.18929853418602668
### iteration step :  40  rmse :  0.09530359507211134
### iteration step :  50  rmse :  0.06411435821480614
### iteration step :  60  rmse :  0.0513655133351369
### iteration step :  70  rmse :  0.045353587569104015
### iteration step :  80  rmse :  0.04211834005578717
### iteration step :  90  rmse :  0.040102228861186175
### iteration step :  100  rmse :  0.03866765487948536
### iteration step :  110  rmse :  0.037540499914071296
### iteration step :  120  rmse :  0.0365947058929847
### iteration step :  130  rmse :  0.03576753856130791
### iteration step :  140  rmse :  0.03502506546496307
### iteration step :  150  rmse :  0.03434737169276892
### iteration step :  160  rmse :  0.033721848888093785
### iteration step :  170  rmse :  0.033139957246185674
### iteration step :  

,course,pred_score
1,양양쏠비치,0.268556
18,"순천만국가정원,순천만습지,정문식당,동강막창",0.198803
26,"파주마장호수,천마루한식부페",0.194556
27,"목포수가정해물순두부,연희네슈퍼,코롬방제과,주일굴보쌈,홈보이",0.193828
